In [20]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
raw_years = pd.read_csv('raw_data/london_weather_data_1979_to_2023.csv')
journeys_2019 = pd.read_csv('raw_data/Journeys_2019.csv')
journeys_2020 = pd.read_csv('raw_data/Journeys_2020.csv')
journeys_2021 = pd.read_csv('raw_data/Journeys_2021.csv')
journeys_2022 = pd.read_csv('raw_data/Journeys_2022.csv')

In [22]:
models = [journeys_2019, journeys_2020, journeys_2021, journeys_2022]
base_path = "data"
if not os.path.exists(base_path):
    os.makedirs(base_path)
for idx, df in enumerate(models):
    year = 2019 + idx
    df['data'] = pd.to_datetime(df['TravelDate'].astype(str), format='%Y%m%d')
    df['suma'] = df['TubeJourneyCount'] + df['BusJourneyCount']
    new_df = df[['data', 'suma']].copy()
    new_df['data_str'] = new_df['data'].dt.strftime('%Y-%m-%d')
    output_path = f"{base_path}/journeys_{year}.csv"
    new_df[['data_str', 'suma']].rename(columns={'data_str': 'data'}).to_csv(output_path, index=False)
    print(f"Zapisano do {output_path}")


Zapisano do data/journeys_2019.csv
Zapisano do data/journeys_2020.csv
Zapisano do data/journeys_2021.csv
Zapisano do data/journeys_2022.csv


In [23]:
base_path = "data"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for year in range(2019, 2023):
    year_start = year * 10000
    year_end = (year + 1) * 10000
    selected_year = raw_years[(raw_years.iloc[:,0] >= year_start) & (raw_years.iloc[:,0] < year_end)]
    selected_year['data'] = pd.to_datetime(selected_year.iloc[:,0].astype(str), format='%Y%m%d')
    tx_col = 'TX' if 'TX' in selected_year.columns else selected_year.columns[1]
    rr_col = 'RR' if 'RR' in selected_year.columns else selected_year.columns[2]
    weather_df = selected_year[['data', tx_col, rr_col]].copy()
    weather_df.columns = ['data', 'TX', 'RR']
    output_path = f"{base_path}/weather_{year}.csv"
    weather_df.to_csv(output_path, index=False)
    print(f"Zapisano dane pogodowe dla roku {year} do {output_path}")


Zapisano dane pogodowe dla roku 2019 do data/weather_2019.csv
Zapisano dane pogodowe dla roku 2020 do data/weather_2020.csv
Zapisano dane pogodowe dla roku 2021 do data/weather_2021.csv
Zapisano dane pogodowe dla roku 2022 do data/weather_2022.csv


C:\Users\lenovo\AppData\Local\Temp\ipykernel_17068\3565145429.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_year['data'] = pd.to_datetime(selected_year.iloc[:,0].astype(str), format='%Y%m%d')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_17068\3565145429.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_year['data'] = pd.to_datetime(selected_year.iloc[:,0].astype(str), format='%Y%m%d')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_17068\3565145429.py:9: SettingWithCopyWarning: 
A val